В прошлый раз мы обсудили, что не хотим брать методы, в которых корректируется функция потерь или изменяется какая-либо архитектура модели(помню, что ты говорил, что хотел бы детекцию): таким образом, остаются следующие классы методов:

(эту разбивку взял из медицинской статьи про noisy labels (https://arxiv.org/pdf/1912.02911.pdf), в ней больше детекционных методов, в общем обзоре(https://arxiv.org/pdf/2007.08199.pdf) практически все методы так или иначе связаны с коррекцией функции потерь или изменением архитектуры)

* **Label cleaning and preprocessing - убираем или заменяем примеры, где предположительно ошибочный лейбл**
* **data reweighting - даем меньшие веса примерам, у которых предположительно ошибочный лейбл**
* **training procedures - изменяем процедуру обучения модели**

Мы договорились о трех-четырех методов, я думаю стоит взять **1-2 простых в реализации метода, 1 средний и 1 сложный метод**

Далее после номера я буду указывать в скобках букву, чтобы указать, что я считаю сложным методом, что простым и средним.
* (e) - easy
* (m) - medium
* (h) - hard


Теперь по методам (у них есть своя нумерация, она показывает из какого параграфа статьи я взял информацию, чтобы было проще понять, что имеется ввиду)(еще я оставил методы в тех же категориях, где они расположены в статье, по описанию я бы сказал, что для некоторых методов нужно менять их расположение)
1. **Label cleaning and preprocessing** - убираем или заменяем примеры, где предположительно ошибочный лейбл
   1. **(m)Train another CNN on clean dataset to clean noisy labels**
   2. **(m) Cleannet - extract feature vector and compare it with a feature vector of it representative class**
   3. **(m) Rank Pruning-identifies data points with confident labels and updates the classifier using only those data points**
   4. **(m) Use GANS to generate a training dataset from initial dataset**
2. **data reweighting** - даем меньшие веса примерам, у которых предположительно ошибочный лейбл
   1. **(m) Metalearning - learn model to give weights to samples**
   2. **(e) Use training strategy-remove a fraction of data for which the loss is the largest**
   3. **(m) Use information about accuracy of annotator**
3. **training procedures** - изменяем процедуру обучения модели
   1. **(m) Cirriculum learning - learn first on less complex and difficult examples**
   2. **(m) Distillation approach - use combination of noisy label and model prediction**
   3. **(h) Co-teaching - Train two identical models and update weights only if label differs**
   4. **(m) Selecting the data samples with correct labels using cross-validation - if predicted and noisy labels agreed, use in another training epoch**
   5. **(e) Learn with clean and noisy dataset with different hyperparameters**
   6. **(h) Mixup - synthesizes new training data points and labels via a convex combination of pairs of training data points and their labels**
   
   
Из этого списка я бы выбрал следующие методы:
Из простых:
* **2.B - Use training strategy-remove a fraction of data for which the loss is the larges** - достаточно дешевая эвристика, думаю ее можно быстро реализовать и попробовать
* **3.E - Learn with clean and noisy dataset with different hyperparameters** - тоже достаточно дешевый метод, делим датасет на чистый (где будут лейблы ассесоров) и на noisy датасет(где оценки толокеров), по разному пробуем обучать модель, с сеткой гиперпараметров

Из средних:
* **3.B - Distillation approach - use combination of noisy label and model prediction** - по моему достаточно интересный метод + у нас что то похожее сейчас внедряется, есть предложение заменять один ответ толокера натренированной моделью, поэтому думаю этот метод даст хороший результат

Из сложных:
* **3.С - Co-teaching - Train two identical models and update weights only if label differs** - это единственный метод, который описывается в статье с общим обзоров методов, как я понимаю, самый навороченный и сложный + по нему вроде как есть торчовая реализация

У нас остается 4 недели, в таком случае план такой:
* **1 неделя** - подготавливаю датасеты, подготавливаю модели для теста, подготавливаю среду для запуска, пробую запускать модели у нас, пишу первую 1 главу и пробую реализовать простые методы
* **2 неделя** - реализую средний по сложности метод, закрываю долги по первой неделе(она получается очень большая по обьему)
* **3 неделя** - реализую сложный метод
* **4 неделя** (там только 4 дня, 5 дней если по 24 включительно) - дооформляем работу, закрываем что не успели за прошлые недели


План работы такой(рассчитываем на 30 страниц):
1. Глава - проблематика и описание методов с датасетом
 1. Вступление, где описывается проблематика noisy labels 
 2. Описание методов, которыми можно бороться с noisy labels
 3. Описание используемых датасетов
2. Глава - описание выбранных методов
  1. Описание простых методов
  2. Описание среднего метода
  3. Описание сложного метода
3. Глава - результаты использования методов
  1. Делаю бенчмарки по тренировке без использования методов
  4. Результаты по использованию простых методов
  5. Результаты по использованию среднего метода
  6. Результаты по использованию сложного метода
4. Вывод

2 и 3 главу может стоит обьединить, просто у нас обычно было деление на 3 главы, поэтому мне привычнее 3.

Да, получается я немного ушел из детекции noisy labels в другую немножко область, но вроде такой обьем выглядит, что его возможно выполнить.

Тогда может завтра обсудить этот план по зуму или договориться здесь, мне сейчас будет необходим пример, с которого я смогу писать работу(чтобы понимать, какой уровень графиков нужен, как ссылки оформлять и т.д.)

Еще вопрос, сколько в работе должно быть ссылок на другие работы? или это некритично?


## 1. Label cleaning and preprocessing

### A. Train another CNN on clean dataset to clean noisy labels

This can be done either prior to training or iteratively in parallel with the training of the main model. [112] proposed supervised and unsupervised image ranking methods for identifying correctly-labeled images in a large corpus of images with noisy labels. The proposed methods were based on matching each image with a noisy label to a set of representative images with clean labels. This method improved the classification accuracy by 4-6% over the baseline CNN models on three datasets. [74] trained two CNNs in parallel using a small dataset with correct labels and a large dataset with noisy labels. The two CNNs shared the feature extraction layers. One CNN used the clean dataset to learn to clean the noisy dataset, which was used by the other CNN to learn the main classification task. Experiments showed that this training method was more effective than training on the large noisy dataset followed by fine-tuning on the clean dataset. [70] trained an ensemble of classifiers on data with noisy labels using cross-validation and used the predictions of the ensemble as soft labels for training the final classifier. 

### B. Cleannet - extract feature vector and compare it with a feature vector of it representative class

CleanNet, proposed by [72], extracts a feature vector from a query image with a noisy label and compares it with a feature vector that is representative of its class. The representative feature vector for each class is computed from a small clean dataset. The similarity between these feature vectors is used to decide whether the label is correct. Alternatively, this similarity can be used to assign weights to the training samples, which is the method proposed for image classification by [72]. [113] improved upon CleanNet in several ways. Most importantly, they removed the need for a clean dataset by estimating the correct labels in an iterative framework. Moreover, they allowed for multiple prototypes (as opposed to only one in CleanNet) to represent each class. Both of these studies reported improvements in image classification accuracy of 1-5% depending on the dataset and noise level. 

### C. Rank Pruning-identifies data points with confident labels and updates the classifier using only those data points

A number of proposed methods for label denoising are based on classification confidence. Rank Pruning, proposed by [73], identifies data points with confident labels and updates the classifier using only those data points. This method is based on the assumption that data samples for which the predicted probability is close to one are more likely to have correct labels. However, this is not necessarily true. In fact, there is extensive recent work showing that standard deep learning models are not “well calibrated” [114], [115]. A classifier is said to have a calibrated prediction confidence if its predicted class probability indicates its likelihood of being correct. For a perfectly-calibrated classifier, P ypredicted = ytrue|pˆ = p  = p. It has been shown that deep learning models produce highly over-confident predictions. Many studies in recent years have aimed at improving the calibration of deep learning models [116], [117], [118]. In order to reduce the reliance on classifier calibration, the Rank Pruning algorithm, as its name suggests, ranks the data samples based on their predicted probability and removes the data samples that are least confident. In other words, Rank Pruning assumes that the predicted probabilities are accurate in the relative sense needed for ranking. In light of what is known about poor calibration of deep learning models, this might still be a strong assumption. Nonetheless, Rank Pruning was shown empirically to lead to substantial improvements in image classification tasks in the presence of strong label noise. Identification of incorrect labels based on prediction confidence was also shown to be highly effective in extensive experiments on image classification by [119], improving the classification accuracy on CIFAR-10 by up to 20% in the presence of very strong label nosie. [120] proposed an iterative label noise filtering approach based on similar concepts as Rank Pruning. This method estimates prediction uncertainty (using such methods as Deep Ensembles [115] or Monte-Carlo dropout [117]) during training and relabels data samples that are likely to have incorrect labels. 

### D. Use GANS

[124] used a generative model to model labeling of large datasets used in deep learning and proposed a label denoising method under this scenario. [125] proposed a GAN for removing label noise from synthetic data generated to train a CNN. This method was shown to be highly effective in removing label noise and improving the model performance. GANs were used to generate a training dataset with clean labels from an initial dataset with noisy labels by [126]. 

## 2. Data reweighting

### A. Metalearning - learn model to give weights to samples

Broadly speaking, these methods aim at down-weighting those training samples that are more likely to have incorrect labels. [87] proposed to weight the training data using a metalearning approach. That method required a separate dataset with clean labels, which was used to determine the weights assigned to the training data with noisy labels. Simply put, it optimized the weights on the training samples by minimizing the loss on the clean validation data. The authors showed that this weighting scheme was equivalent to assigning larger weights to training data samples that were similar to the clean validation data in terms of both the learned features and optimization gradient directions. Experiments showed that this method improved upon baseline methods by 0.5% and 3% on CIFAR-10 and CIFAR-100 with only 1000 images with clean labels. More recently, [142] proposed to re-weight samples by optimization gradient re-scaling. The underlying idea, again, is to give larger weights to samples that are easier to learn, hence more likely to have correct labels. Pumpout, proposed by [143], is also based on gradient scaling. The authors propose two methods for identifying data samples that are likely to have incorrect lables. One of their methods is based on the assumption that data samples with incorrect labels are likely to display unusually high loss values. Their second method is based on the value of the backward-corrected loss [139]; they suggest that the condition 1 T T −1 l(ˆp(y|x)) < 0 indicates data samples with incorrect labels. For training data samples that are suspected of having incorrect labels, the gradients are scaled by −γ, where 0 < γ < 1. In other words, they perform a scaled gradient ascent on the samples with incorrect labels. In several experiments, including image classification with MNIST and CIFAR-10 datasets, they show that their method avoids fitting to incorrect labels and reduces the classification error by up to 40%.

### B. Training strategy-remove a fraction of data for which the loss is the largest

[95] proposed a training strategy that can be interpreted as a form of data re-weighting. In each training epoch, they remove a fraction of the data for which the loss is the largest, and update the model parameters to minimize the loss function on the remaining training data. This method assumes that the model gradually converges towards a good classifier such that the mis-labeled training samples exhibit unusually high loss values as training progresses. The authors proved that this simple approach learns the optimal model in the case of generalized linear models. For deep CNNs that are highly nonlinear, they empirically showed the effectiveness of their method on several image classification tasks. As in the case of this method, there is often a close connection between some of the data re-weighting methods and methods based on robust loss functions. [92] built upon this connection and developed it further by proposing to learn a data re-weighting scheme from data. Instead of assuming a pre-defined weighting scheme, they used a multi-layer perceptron (MLP) model with a single hidden layer to learn a suitable weighting strategy for the task and the dataset at hand. The MLP in this method is trained on a small dataset with clean labels. Experiments on datasets with 8 unbalanced and noisy labels showed that the learned weighting scheme conformed with those proposed in other studies. Specifically, for data with noisy labels the model learned to down-weight samples with large loss functions, the opposite of the form learned for datasets with unbalanced classes. One can argue that this observation empirically justifies the general trend towards down-weighting training samples with large loss values when training with noisy labels. 

### C. Use accuracy of annotator

A common scenario involves labels obtained from multiple sources or annotators with potentially different levels of accuracy. This is a heavily-researched topic in machine learning. A simple approach to tackling this scenario is to use expectationmaximization (EM)-based methods such as [144], [145] to estimate the true labels and then proceed to train the deep learning model using the estimated labels. [93] proposed an iterative method, whereby model predictions were used to estimate annotator accuracy and then these accuracies were used to train the model with a loss function that properly weighted the label from each annotator. The model was updated via gradient descent, whereas annotator confusion matrices were optimized with an EM method. By contrast, [94] estimated the network weights as well as annotator confusion matrices via gradient descent. 

## 3. training procedures

### A. Cirriculum learning - learn first on less complex and difficult examples

Several methods based on curriculum learning have been proposed to combat label noise. Curriculum learning, first proposed by [148], is based on training a model with examples of increasing complexity or difficulty. In the method proposed by [107], an LSTM network called Mentor-Net provides a curriculum, in the form of weights on the training samples, to a second network called Student-Net. On CIFAR-100 and ImageNet with various label noise levels, their method improved 9 the classification accuracy by up to 20% and 2%, respectively. [149] proposed another method based on curriculum learning, named CurriculumNet, for training a model from massive datasets with noisy labels. This method first clusters the training data in some feature space and identifies samples that are more likely to have incorrect labels as those that fall in low-density clusters. The data are then sequentially presented to the main CNN model to be trained. This technique achieved good results on several datasets including ImageNet. The SelfError-Correcting CNN proposed by [150] is based on similar ideas; the training begins with noisy labels but as the training proceeds the network is allowed to change a sample’s label based on a confidence policy that gives more weight to the network predictions with more training. 

### B. Distillation approach - use combination of noisy label and model prediction

[151] adopted a knowledge distillation approach [152] to train an auxiliary model on a small dataset with clean labels to guide the training of the main model on a large dataset with noisy labels. In brief, their approach amounts to using a pseudo-label, which is a convex combination of the noisy label and the label predicted by the auxiliary model. To reduce the risk of overfitting the auxiliary model on the small clean dataset, the authors introduced a knowledge graph based on the label transition matrix. [101] also proposed using a convex combination of the noisy labels and labels predicted by the model at its current training stage. They suggested that as the training proceeds, the model becomes more accurate and its predictions can be weighted more strongly, thereby gradually forgetting the original incorrect labels. [102] used a similar approach for face identification. They first trained their model on a small dataset with less label noise and then fine-tuned it on data with stronger label noise using an iterative label update strategy similar to that explained above. Their method led to improvements of up to 2% in face recognition accuracy. Following a similar training strategy, [120] suggested that there is a point (e.g., a training epoch) when the model learns the true data features and is about to fit to the noisy labels. They proposed two methods, one based on the predictions on a clean dataset and another based on prediction uncertainty measures, to identify that stage in training. The output of the model at that stage can be used to fix the incorrect labels. 

### C. Co-teaching - Train two identical models and update weights only if label differs

A number of studies have proposed methods involving joint training of more than one model. For example, one work suggested simultaneously training two separate but identical networks with random initialization, and only updating the network parameters when the predictions of the two networks differed [153]. The idea is that when training with noisy labels, the model starts by learning the patterns in data samples with correct labels. Later in training, the model will struggle to overfit to samples with incorrect labels. The proposed method hopes to reduce the impact of label noise because the decision as to whether or not to update the model is made based on the predictions of the two models and independent of the noisy label. In other words, on data with incorrect labels both models are likely to produce the same prediction, i.e., they will predict the correct label. On easy examples with correct labels, too, both models will make the same (correct) prediction. On hard examples with correct labels, on the other hand, the two models are more likely to disagree. Hence, with the proposed training strategy, the data samples that will be used in later stages of training will shrink to the hard data samples with correct labels. This strategy also improves the computational efficiency since it performs many updates at the start of training but avoids unnecessary updates on easy data samples once the models have sufficiently converged to predict the correct label on those samples. This idea was developed into co-teaching [108], whereby the two networks identified label-noise-free samples in their mini-batches and shared the update information with the other network. The authors compare their method with several state of the art techniques including Mentor-Net ([107]. Their method outperformed competing methods in most experiments, while narrowly underperforming in some experiments. Co-teaching was further improved in [154], where the authors suggested to focus the training on data samples with lower loss values in order to reduce the risk of training on data with incorrect labels. Along the same lines, [155] proposed a meta-learning objective that encouraged consistent predictions between a student model trained on noisy labels and a teacher model trained on clean labels. The goal was to train the student model to be tolerant to label noise. Towards this goal, artificial label noise was added on data with correct labels to train the student model. The student model was encouraged to be consistent with the teacher model using a meta-objective in the form of the KL divergence between prediction probabilities. Their method outperformed several competing methods by 1-2% on CIFAR-10 and Clothing1M datasets. 

### D. Selecting the data samples with correct labels using cross-validation

Experiments in [62] showed that co-teaching was less effective as the label noise increased. Instead, the authors showed that selecting the data samples with correct labels using cross-validation was more effective. In their proposed approach, the training data was divided into two folds. The model was iteratively trained on one fold and tested on the other. Data samples for which the predicted and noisy labels agreed were assumed to have the correct label and were used in the next training epoch. One study proposed to learn the network parameters by optimizing the joint likelihood of the network parameters and true labels [156]. Compared with standard training with cross-entropy loss, this method improved the classification accuracy on CIFAR-10 by 2% with low label noise rate to 17% when label noise rate was very high. 

### E. - Learn with clean and noisy dataset with different hyperparameters

Some studies have suggested modifying the learning rate, batch size, or other settings in the training methodology. For example, for applications where multiple datasets with varying levels of label noise are available, [157] have proposed training strategies in terms of the order of using different datasets during training and proper learning rate adjustments based on the level of label noise in each dataset. Assuming that separate clean and noisy datasets are available, the same study has shown that using different learning rates for training with noisy and clean samples can improve the performance. It has also shown that the optimal ordering of using the two datasets (i.e., whether to train on the noisy dataset or the clean dataset first) depends on the choice of the learning rate. It has also been suggested that when label noise is strong, the effective batch size decreases, and that batch size should be increased 10 with a proper scaling of the learning rate [61]. [76] proposed to include samples from a noisy dataset and a clean dataset in each training mini-batch, giving higher weights to the samples with clean labels. 

### D. Mixup - synthesizes new training data points and labels via a convex combination of pairs of training data points and their labels

Mixup is a less intuitive but simple and effective method [109]. It synthesizes new training data points and labels via a convex combination of pairs of training data points and their labels. More specifically, given two randomly selected training data and label pairs (xi , yi) and (xj , yj ), a new training data point and label are synthesized as x˜ = λxi + (1 − λ)xj and y˜ = λyi + (1 − λ)yj , where λ ∈ [0, 1] is sampled from a beta distribution. Although mixup is known primarily as a data augmentation and regularization strategy, it has been shown to be remarkably effective for combatting label noise. Compared with basic emprirical risk minimization on CIFAR10 dataset with different levels of label noise, mixup reduced the classification error by 6.5-12.5%. The authors argue that the reason for this behavior is because interpolation between datapoints makes memorization on noisy labels, as observed in [59], more difficult. In other words, it is easier for the network to learn the linear iterpolation between datapoints with correct labels than to memorize the interploation when labels are incorrect. The same idea was successfully used in video classification by [70]. 